In [3]:
### hiển thị tất cả file trong folder message lên trên left_frame


import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
def browser():
    folder_path = filedialog.LoadFileDialog(root)

button_browse = tk.Button(root, text='Browser', command = browser)
button_browse.grid(row=0, column=2, padx=(5, 10), pady=5)

root.mainloop()

: 

In [3]:
import tkinter as tk
from tkinter import filedialog
from tkinter.ttk import Progressbar
from tkinter import messagebox
import threading
import g4f
import os


main_window_width = 1200
main_window_height = 500

left_frame_background_color = '#202123'
right_frame_background_color = '#444654'


top_right_frame_background_color = '#444654'
bottom_right_frame_background_color = '#404040'

fg_loading_label = 'black'
bg_loading_label = '#606060'

text_color = '#444654'

loading_flag = False
file_path = None
loading_flag = False
created = False


def center_window(window, width, height): # đừng động vào cái này !!!!!!!
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    x = (screen_width - width) // 2
    y = (screen_height - height) // 2
    window.geometry(f"{width}x{height}+{x}+{y}")

class Main_Window():
    def __init__(self, window):
        self.window = window
        window.resizable(True, True)
        self.left_frame = tk.Frame(window, width= main_window_width/4 , height=main_window_height, bg=left_frame_background_color)# Phần mành hình hiển thị các file.
        self.left_frame.pack(side="left",fill='both',expand= True)

        self.right_frame = tk.Frame(window, width= main_window_width - main_window_width/4 , height=main_window_height, bg=right_frame_background_color)
        self.right_frame.pack(side="right",fill='both', expand= True)

        self.top_right_frame = tk.Frame(self.right_frame, width= main_window_width - main_window_width/3 , height=main_window_height//2 , bg=top_right_frame_background_color)
        self.top_right_frame.pack_propagate(True)
        self.top_right_frame.pack(side = 'top', fill='both', expand=True)  # Fill và expand để frame mở rộng theo kích thước của parent

        self.top_right_text = tk.Text(self.top_right_frame, wrap=tk.WORD, state=tk.DISABLED, bg= text_color)
        self.top_right_text.pack(expand=True, fill='both')

        self.bottom_right_frame = tk.Frame(self.right_frame, width=main_window_width - main_window_width/3, height=main_window_height//2, bg=bottom_right_frame_background_color)
        self.bottom_right_frame.pack_propagate(True)
        self.bottom_right_frame.pack(side= 'bottom', fill='both', expand=True)

        self.entry = tk.Entry(self.bottom_right_frame, width=70, font=('Arial', 18))
        self.entry.pack(side='bottom', pady=10, padx=10)

        self.submit_button = tk.Button(self.bottom_right_frame, text="Submit", command= self.get_entry)
        self.submit_button.pack(side='right')

        self.loading_label = tk.Label(self.top_right_frame, text="Loading...", fg = fg_loading_label, bg = bg_loading_label)  # Đổi màu và font chữ
        self.loading_label.pack(side="left", fill="both", expand=True, padx=10, pady=10)

        self.progress_bar = Progressbar(self.top_right_frame, mode='indeterminate', length = 83)
        self.progress_bar.pack(side="right", fill="both", expand=True, padx=10, pady=10)

    def display_message(self,message, sender): # chức năng hiện chữ lên màn hình.
        global file_path
        self.top_right_text.config(state=tk.NORMAL, fg="#FFFFFF")  # Cho phép cập nhật nội dung

        with open(str(file_path), 'a', encoding='utf-8') as file: #
            if sender == "user":
                self.top_right_text.insert(tk.END, f"You: {message}\n\n", "user_message")
                file.write(f"You: {message}\n\n")
            elif sender == "bot":
                self.top_right_text.insert(tk.END, f"Bot: {message}\n\n", "bot_message")
                file.write(f"Bot: {message}\n\n")

        self.top_right_text.config(state=tk.DISABLED)  # Khóa trạng thái
        self.top_right_text.see(tk.END)

    def process_message(self,user_message):
        global loading_flag
        self.window.after(0, lambda: self.loading_label.config(background='green'))  # Thay đổi màu của label khi bắt đầu loading
        response = g4f.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": user_message}],
            stream=True,
        )
        ans = ""
        for text in response:
            ans += text
        loading_flag = False
        self.window.after(0, lambda: self.display_message(ans, "bot"))  # Hiển thị tin nhắn của bot
        self.window.after(0, lambda: self.progress_bar.stop())
        self.window.after(0, lambda: self.loading_label.config(background="white"))

    def get_entry(self,event=None):
        global loading_flag,created
        if created:
            request = self.entry.get()
            self.display_message(request, "user")
            self.entry.delete(0, tk.END)

            loading_flag = True
            self.progress_bar.start()

            thread = threading.Thread(target= self.process_message, args=(request,))
            thread.start()
        else:
            messagebox.showerror("Warning", "You must create the file.txt first")
class Login_window(Main_Window):
    def __init__(self,frame):
        self.login_frame = frame

        # Tạo và thiết lập các thành phần giao diện
        self.label_folder = tk.Label(self.login_frame, text='Folder Location')
        self.entry_folder = tk.Entry(self.login_frame, width=50)
        self.button_browse = tk.Button(self.login_frame, text='Browser', command= self.browse_folder)

        self.label_file = tk.Label(self.login_frame, text='Filename.txt')
        self.entry_file = tk.Entry(self.login_frame, width=50)

        self.button_create = tk.Button(self.login_frame, text='Save',padx= 10, command= self.create_file)
        self.button_clear = tk.Button(self.login_frame, text='Back',padx= 10, command= self.clear_fields)

        self.result_label = tk.Label(self.login_frame, text='', font=('Arial', 10, 'italic'))

        # Định vị các thành phần giao diện
        self.label_folder.grid(row=0, column=0, padx=(10, 5), pady=5, sticky='e')
        self.entry_folder.grid(row=0, column=1, padx=5, pady=5)
        self.button_browse.grid(row=0, column=2, padx=(5, 10), pady=5)

        self.label_file.grid(row=1, column=0, padx=(10, 5), pady=5, sticky='e')
        self.entry_file.grid(row=1, column=1, padx=5, pady=5)

        self.button_create.grid(row=2, column=0, columnspan=2, pady=(10, 5))
        self.button_clear.grid(row=2, column=1, columnspan=2, pady=(10, 5))

        self.result_label.grid(row=3, column=0, columnspan=3, pady=(0, 10))

        self.login_frame.protocol("WM_DELETE_WINDOW", self.on_root_close)

        self.login_frame.resizable(False,False)

    def create_file(self):
        global created, file_path

        folder_path = self.entry_folder.get()
        file_name = self.entry_file.get()

        if folder_path and file_name:
            file_path = os.path.join(folder_path, file_name)

            try:
                with open(str(file_path), 'w',encoding='utf-8') as file: #
                    pass  # Tạo một file trống
                self.result_label.config(text=f'File {file_name} đã được tạo thành công trong {folder_path}')
                created = True 
                self.login_frame.after(1000, self.login_frame.destroy) # tắt cửa sổ.
            except Exception as e:
                self.result_label.config(text=f'Lỗi: {str(e)}')
        else:
            self.result_label.config(text='Hãy điền đầy đủ đường dẫn và tên file.')

    def browse_folder(self):
        folder_path = filedialog.askdirectory()
        self.entry_folder.delete(0, tk.END)
        self.entry_folder.insert(0, folder_path)

    def clear_fields(self):
        self.entry_folder.delete(0, tk.END)
        self.entry_file.delete(0, tk.END)
        self.result_label.config(text='')

    def on_root_close(self):
        if not created:
            self.login_frame.destroy() #Đóng cửa sổ tạo file
            self.window.destroy()  # Đóng cửa sổ chat chính
root = tk.Tk()
create_file_window = Login_window(root)
center_window(root,550,140)
root.mainloop() 

mainwin = tk.Tk()
main_window = Main_Window(mainwin)
center_window(mainwin,main_window_width,main_window_height)
mainwin.mainloop()

ModuleNotFoundError: No module named 'g4f'